In [1]:
from datetime import datetime
from datetime import timedelta

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import string
import nltk 
import re
%matplotlib inline

In [3]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

In [11]:
df = pd.read_csv("bTwitter11.csv")

In [5]:
df['timestamp'] = pd.to_datetime(df['timestamp'],format="%Y-%m-%d %H:%M:%S")
df =df.set_index('timestamp')

In [13]:
sw = stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in sw]
    return text
ps = nltk.PorterStemmer()
def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

wl = nltk.WordNetLemmatizer()
def lemmatizer(text):
    text = [wl.lemmatize(word) for word in text]
    return text

In [14]:
df['tags_token'] = df['all_hashtags_seg'].apply(lambda x: word_tokenize(x.lower()))
df['tags_nonstop'] = df['tags_token'].apply(lambda x: remove_stopwords(x))
df['tags_stem'] = df['tags_nonstop'].apply(lambda x: stemming(x))
df['tags_lem'] = df['tags_stem'].apply(lambda x: lemmatizer(x))

In [15]:
def tagstr(s):
    return ' '.join(s)

In [16]:
df['tags'] =  df['tags_nonstop'].apply(lambda x: tagstr(x)) 

In [17]:
df.head()

,timestamp,hashtags,tweets,replies,retweets,likes,username,location,friends,followers,verified,all_hashtags,all_hashtags_seg,tags_token,tags_nonstop,tags_stem,tags_lem,tags
0,2020-11-01 23:59:19,peace,To all whom I loved in this #life and to all t...,NaN,NaN,NaN,metsbookworm973,"Inverness, FL",600,520,0,life AllSaintsDay pray peace God Father Heaven,life all saints day pray peace godfather heaven,"[life, all, saints, day, pray, peace, godfathe...","[life, saints, day, pray, peace, godfather, he...","[life, saint, day, pray, peac, godfath, heaven]","[life, saint, day, pray, peac, godfath, heaven]",life saints day pray peace godfather heaven
1,2020-11-01 23:51:15,peace,You can't be strong all the time. Sometimes yo...,NaN,NaN,NaN,Zzaqwan02,NaN,4,3,0,peace,peace,[peace],[peace],[peac],[peac],peace
2,2020-11-01 23:46:27,peace,Meu teto é o céu!!\n#chapadadosveadeiros #sky ...,NaN,NaN,1,valtissimos,Goiânia - Goiás - Brasil,290,104,0,chapadadosveadeiros sky sol peace vibes goodvibes,chapada dosveadeirosskysol peace vibes good vibes,"[chapada, dosveadeirosskysol, peace, vibes, go...","[chapada, dosveadeirosskysol, peace, vibes, go...","[chapada, dosveadeirosskysol, peac, vibe, good...","[chapada, dosveadeirosskysol, peac, vibe, good...",chapada dosveadeirosskysol peace vibes good vibes
3,2020-11-01 23:41:04,peace,#cubans #Latinos #Hispanics #blackvoter this #...,3.0,1,1,plzhelpkids,"Anywhere, usa",900,157,0,cubans Latinos Hispanics blackvoter ElectionDa...,cubans latinos hispanics black voter election ...,"[cubans, latinos, hispanics, black, voter, ele...","[cubans, latinos, hispanics, black, voter, ele...","[cuban, latino, hispan, black, voter, elect, d...","[cuban, latino, hispan, black, voter, elect, d...",cubans latinos hispanics black voter election ...
4,2020-11-01 23:38:00,peace,Teach #peace,NaN,NaN,1,RODRIGObessa,"Belo Horizonte, Brasil","4,557","3,926",0,peace,peace,[peace],[peace],[peac],[peac],peace


In [18]:
from sklearn.feature_extraction.text import CountVectorizer
# the vectorizer object will be used to transform text to vector form
vectorizer = CountVectorizer()
# apply transformation
tf = vectorizer.fit_transform(df['tags'])#).toarray()
# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names()

In [19]:
from sklearn.decomposition import LatentDirichletAllocation
number_of_topics = 10
model = LatentDirichletAllocation(n_components=number_of_topics, random_state=45) # random state for reproducibility
# Fit data to model
model.fit(tf)

LatentDirichletAllocation(random_state=45)

In [20]:
n_top_words = 10
for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([tf_feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

Topic #0: peace zen yoga war quotes buddha art wisdom love mindfulness
Topic #1: conflict war history politics climate china truth world justice military
Topic #2: azerbaijan armenia war karabakh nagorno conflict artsakh peace armenian russia
Topic #3: peace love god jesus joy conflict positive hope faith bible
Topic #4: war stop artsakh turkey peace day erdogan conflict remembrance terrorist
Topic #5: peace conflict war team love rights management human world psychic
Topic #6: conflict peace vote united congress war states america day freedom
Topic #7: peace war conflict music love life film protest action writing
Topic #8: peace election war love 2020 trump biden humanity life vote
Topic #9: peace love nature spiritual music life self meditation unity health


In [22]:
from collections import defaultdict
def generate_ngrams(text, n_gram=1):
    token = [token for token in text.lower().split(' ') if token != '' ]
    ngrams = zip(*[token[i:] for i in range(n_gram)])
    return [' '.join(ngram) for ngram in ngrams]

N = 100

# Bigrams
text_bigrams = defaultdict(int)

for text in df['all_hashtags_seg']:
    for word in generate_ngrams(text, n_gram=2):
        text_bigrams[word] += 1

df_bigrams = pd.DataFrame(sorted(text_bigrams.items(), key=lambda x: x[1])[::-1])
df_bigrams.columns = ['bigrams','count']
df_bigrams.head(20)

,bigrams,count
0,love peace,109
1,stop erdogan,93
2,war artsakh,81
3,artsakh republic,78
4,peace love,76
5,turkey stop,72
6,terrorist turkey,69
7,republic terrorist,68
8,defend artsakh,62
9,erdogan defend,61
